In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd
import csv
import sys

from sklearn.metrics import mean_squared_error
from generative_model import GenerativeModel as MyGM

In [2]:
# X_train_path  = 'data/format/X_train'
# Y_train_path  = 'data/format/Y_train'
# X_test_path  = 'data/format/X_test'

# text = open(X_train_path, 'r') 
# row = csv.reader(text , delimiter=",")
# X_train = []
# for r in row:
#     X_train.append(r)

# text = open(Y_train_path, 'r') 
# row = csv.reader(text , delimiter=",")
# Y_train = []
# for r in row:
#     Y_train.append(r)

# text = open(X_test_path, 'r') 
# row = csv.reader(text , delimiter=",")
# X_test = []
# for r in row:
#     X_test.append(r)

# train_X = (np.array(X_train)[1:,]).astype("float")
# train_y = np.array(Y_train[1:]).flatten().astype("float")
# test_X = (np.array(X_test)[1:,]).astype("float")

In [3]:
class GenerativeModel(object):
    def __init__(self):
        pass
        
    def fit(self, X,y):
        ####### find mean, coveriance matrix of two class ######
        # seperate y=0 and y=1
        self.c1_ind = np.where(y==1)[0]
        self.c2_ind = np.where(y==0)[0]
        
        # compute stastistic
        self.c1_X = X[self.c1_ind,:]
        self.c2_X = X[self.c2_ind,:]
        self.u1 = np.mean(self.c1_X ,axis=0)
        self.u2 = np.mean(self.c2_X ,axis=0)
        self.p1 = len(self.c1_ind)/(len(self.c1_ind)+len(self.c2_ind))
        self.p2 = 1 - self.p1
        self.cov =  np.cov(self.c1_X,rowvar=False) * self.p1 + np.cov(self.c2_X,rowvar=False) * self.p2
        self.cov_inv = np.linalg.pinv(self.cov)
        
    def predict(self, test_X):
        z =((test_X).dot(self.cov_inv).dot(self.u1-self.u2)- 
            (1/2)*(self.u1).dot(self.cov_inv).dot(self.u1)+ (1/2)*(self.u2).dot(self.cov_inv).dot(self.u2)
            +np.log(len(self.c1_ind)/len(self.c2_ind)))
        
        # print(f"z.shape = {z.shape}")
        
        # return self.sigmoid(z)
        
        return self.sign(self.sigmoid(z))
    
    def sigmoid(self, z):
        res = 1 / (1.0 + np.exp(-z))
        return np.clip(res, 1e-9, 1-(1e-9))
    
    def sign(self, a):
        print(a)
        output = []
        for i in a:
            if i>0.5:
                output.append(1)
            else:
                output.append(0)
                
        return np.array(output)

In [4]:
train_x_csv_path = 'data/format/X_train'
train_x_df = pd.read_csv(train_x_csv_path)
train_y_csv_path = 'data/format/Y_train'
train_y_df = pd.read_csv(train_y_csv_path)
test_x_csv_path = 'data/format/X_test'
test_x_df = pd.read_csv(test_x_csv_path)
train_x = train_x_df.values
train_y = train_y_df.values
test_x = test_x_df.values

In [5]:
gm_ref = GenerativeModel()
gm_ref.fit(train_x,train_y)
gm_ref_pred = gm_ref.predict(test_x)
# np.unique(gm_ref_pred, return_counts=True)

[0.0114754  0.11555019 0.37480357 ... 0.73246286 0.15260898 0.89295688]


In [6]:
my_gm = MyGM()
my_gm.fit(train_x, train_y)
my_pred = my_gm.predict(test_x)

In [7]:
# np.mean(my_gm.predict(my_gm.c1))

In [8]:
# np.mean(my_gm.predict(my_gm.c2))

In [9]:
np.array_equal(gm_ref.cov, my_gm.sigma)
np.array_equal(gm_ref.u1, my_gm.mu_c1)
np.array_equal(gm_ref.u2, my_gm.mu_c2)
print(len(gm_ref.c1_ind), my_gm.m_c1)
print(len(gm_ref.c2_ind), my_gm.m_c2)

True

True

True

7841 7841
24720 24720


In [10]:
my_pred

array([0, 0, 0, ..., 1, 0, 1])

In [11]:
gm_ref_pred

array([0, 0, 0, ..., 1, 0, 1])

In [12]:
round(mean_squared_error(my_pred, gm_ref_pred), 6)

6.1e-05

In [13]:
np.array_equiv(my_pred, gm_ref_pred)

False

In [14]:
count = 0
for a, b in zip(my_pred, gm_ref_pred):
    if a != b:
        count += 1
count

1